# Model Controller Tutorial: GPT2 model (Multi Head)

> This notebook contains some example of how to use the GPT2-based models in this NLP library

- skip_showdoc: true
- skip_exec: true

In this series, we walk through some of the capability of this library: single-head classification, multi-head classification, multi-label classification, and regression. If you want a more detailed tutorial, check [this](https://anhquan0412.github.io/that-nlp-library/model_classification_tutorial.html) out

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
from transformers import AutoTokenizer
from datasets import Dataset

# Define the custom augmentation function

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Department Name'],
                         sup_types=['classification','classification']
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         metadatas=['Title'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

Define our tokenizer for GPT2

In [ ]:
_tokenizer = AutoTokenizer.from_pretrained('gpt2')
_tokenizer.pad_token = _tokenizer.eos_token
_tokenizer.padding_side = 'left'

In [ ]:
print(_tokenizer)
print(len(_tokenizer))

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)
50257


Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18101
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

# Model Experiment: GPT2 Vanilla Multihead classification

## Define and train a vanilla GPT2 model

In [ ]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

In [ ]:
from that_nlp_library.models.roberta.classifiers import ConcatHeadSimple
from that_nlp_library.model_main import *
from that_nlp_library.models.gpt2.classifiers import *
from sklearn.metrics import f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


In [ ]:
gpt2body = GPT2Model.from_pretrained('gpt2')

In [ ]:
num_classes = [len(tdc.label_lists[0]),len(tdc.label_lists[1])] 
num_classes

[3, 6]

In [ ]:
# our model is more complex, so it's best to define some of its arguments
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'is_multilabel':tdc.is_multilabel, # False
    'is_multihead':tdc.is_multihead, # True
    'head_weights':[1,1],
    # classfication head hyperparams
    'classifier_dropout':0.1 
}

In [ ]:
model = model_init_classification(model_class = GPT2BaseForSequenceClassification,
                                  cpoint_path = 'gpt2', 
                                  output_hidden_states=False, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=gpt2body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
# resize token embedding
model.body_model.resize_token_embeddings(len(_tokenizer))

Embedding(50257, 768)

In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

And we can start training our model

In [ ]:
lr = 8e-5
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
              )


Epoch,Training Loss,Validation Loss,F1 Score Division name,Accuracy Score Division name,F1 Score Department name,Accuracy Score Department name
1,No log,1.230941,0.417887,0.610252,0.641590,0.859258
2,1.726900,1.192589,0.479904,0.604286,0.669439,0.874945
3,1.726900,1.145920,0.449733,0.613566,0.674166,0.877817


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model1')

## Make predictions

In [ ]:
trained_model = model_init_classification(model_class = GPT2BaseForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model1'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

In [ ]:
df_val = controller.predict_ddict_classification(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Division Name,Department Name,label,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,,. such a fun jacket ! great to wear in the spr...,General Petite,Intimate,"[1, 2]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.531224,Jackets,0.859643
1,simple and elegant,simple and elegant . i thought this shirt was ...,General Petite,Tops,"[1, 4]","[36439, 290, 19992, 764, 1312, 1807, 428, 1014...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.590583,Tops,0.989144
2,retro and pretty,retro and pretty . this top has a bit of a ret...,General,Tops,"[0, 4]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.631959,Tops,0.991444
3,summer/fall wear,summer / fall wear . i first spotted this on a...,General Petite,Dresses,"[1, 1]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",General Petite,0.512305,Dresses,0.949437
4,perfect except slip,perfect except slip . this is my new favorite ...,General Petite,Dresses,"[1, 1]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.528962,Dresses,0.990059


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val['Division Name'],df_val['pred_Division Name'],average='macro')

0.44973285542706803

In [ ]:
f1_score(df_val['Department Name'],df_val['pred_Department Name'],average='macro')

0.6741658713603403

# Model Experiment: GPT2 Multi-Head Classification (with Hidden Layer Concatenation)

In [ ]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

In [ ]:
from that_nlp_library.models.roberta.classifiers import ConcatHeadSimple
from that_nlp_library.model_main import *
from that_nlp_library.models.gpt2.classifiers import *
from sklearn.metrics import f1_score, accuracy_score

## Define and train a custom GPT2 model

In [ ]:
num_classes = [len(tdc.label_lists[0]),len(tdc.label_lists[1])] 
num_classes

[3, 6]

In [ ]:
gpt2body = GPT2Model.from_pretrained('gpt2')

In [ ]:
# our model is more complex, so it's best to define some of its arguments
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    'is_multilabel':tdc.is_multilabel, # False
    'is_multihead':tdc.is_multihead, # True
    'head_weights':[1,1], # weights for label 1 and label 2 This means L2's weight is twice as much as L1's
    # classfication head hyperparams
    'layer2concat':2, # you can change the number of layers to concat (default is 4, based on the paper)
    'classifier_dropout':0.1 
}

model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                  cpoint_path = 'gpt2', 
                                  output_hidden_states=True, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=gpt2body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
metric_funcs = [partial(f1_score,average='macro'),accuracy_score]
controller = ModelController(model,tdc,seed=42)

And we can start training our model

In [ ]:
seed_everything(42)

In [ ]:
lr = 8e-5
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics_classification,
              )

# Epoch	Training Loss	Validation Loss	F1 Score Division name	Accuracy Score Division name	F1 Score Department name	Accuracy Score Department name
# 1	No log	1.230941	0.417887	0.610252	0.641590	0.859258
# 2	1.726900	1.192589	0.479904	0.604286	0.669439	0.874945
# 3	1.726900	1.145920	0.449733	0.613566	0.674166	0.877817

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1 Score Division name,Accuracy Score Division name,F1 Score Department name,Accuracy Score Department name
1,No log,1.522329,0.429663,0.558330,0.571077,0.830093
2,2.117100,1.225006,0.483412,0.587494,0.653936,0.867433
3,2.117100,1.176845,0.460362,0.608263,0.660864,0.871410


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model1')

## Make predictions

### Load trained model

In [ ]:
_model_kwargs

{'head_class_sizes': [3, 6],
 'head_class': that_nlp_library.models.roberta.classifiers.ConcatHeadSimple,
 'is_multilabel': False,
 'is_multihead': True,
 'head_weights': [1, 1],
 'layer2concat': 2,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model1'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

### Predict Train/Validation set

In [ ]:
df_val = controller.predict_ddict_classification(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Division Name,Department Name,label,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,,. such a fun jacket ! great to wear in the spr...,General Petite,Intimate,"[1, 2]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General Petite,0.559493,Jackets,0.909249
1,simple and elegant,simple and elegant . i thought this shirt was ...,General Petite,Tops,"[1, 4]","[36439, 290, 19992, 764, 1312, 1807, 428, 1014...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.525662,Tops,0.987693
2,retro and pretty,retro and pretty . this top has a bit of a ret...,General,Tops,"[0, 4]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.686668,Tops,0.974669
3,summer/fall wear,summer / fall wear . i first spotted this on a...,General Petite,Dresses,"[1, 1]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",General,0.658008,Dresses,0.916244
4,perfect except slip,perfect except slip . this is my new favorite ...,General Petite,Dresses,"[1, 1]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.516651,Dresses,0.977569


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
f1_score(df_val['Division Name'],df_val['pred_Division Name'],average='macro')

0.46075081995523437

In [ ]:
f1_score(df_val['Department Name'],df_val['pred_Department Name'],average='macro')

0.6608640860018918

### Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)
# drop NaN values in the label column
df_test = df_test[~df_test['Department Name'].isna()].reset_index(drop=True)

# save the label, as we will calculate some metrics later. We also filter out labels with NaN Review Text,
# as there will be a filtering processing on the test set
true_labels = df_test.loc[~df_test['Review Text'].isna(),'Department Name'].values 

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop(['Division Name','Department Name'],axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True, # since we have some text filtering in the processing
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.648170,Tops,0.993969
1,,. i don't know why i had the opposite problem ...,"[13, 1312, 836, 470, 760, 1521, 1312, 550, 262...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.695314,Bottoms,0.993493
2,great pants,great pants . thes e cords are great--lightwei...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.542203,Bottoms,0.995058
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[41199, 401, 24928, 329, 257, 4936, 866, 764, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",General,0.715717,Tops,0.902689
4,short and small,short and small . the shirt is mostly a thick ...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",General,0.653726,Tops,0.923175


Let's quickly check the f1 score to make sure everything works correctly

In [ ]:
f1_score(true_labels,df_test_predicted['pred_Department Name'],average='macro')

0.6704494496423042

Predict top k results

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True,
                                                   topk=3
                                                  )

Filter (num_proc=20):   0%|          | 0/4692 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4528 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/4528 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[General, General Petite, Initmates]","[0.6481702, 0.34613168, 0.0056981617]","[Tops, Intimate, Jackets]","[0.9939686, 0.004535006, 0.0008420434]"
1,,. i don't know why i had the opposite problem ...,"[13, 1312, 836, 470, 760, 1521, 1312, 550, 262...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.69531435, 0.29860473, 0.0060809045]","[Bottoms, Intimate, Trend]","[0.99349266, 0.0040003383, 0.00088396604]"
2,great pants,great pants . thes e cords are great--lightwei...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[General, General Petite, Initmates]","[0.54220295, 0.44460025, 0.01319685]","[Bottoms, Intimate, Trend]","[0.9950577, 0.0046746954, 0.0001348941]"
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[41199, 401, 24928, 329, 257, 4936, 866, 764, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[General, General Petite, Initmates]","[0.71571684, 0.25643656, 0.027846621]","[Tops, Jackets, Intimate]","[0.9026886, 0.06395986, 0.021722412]"
4,short and small,short and small . the shirt is mostly a thick ...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[General, General Petite, Initmates]","[0.65372556, 0.2964127, 0.04986173]","[Tops, Intimate, Jackets]","[0.92317545, 0.06723344, 0.0073472783]"


In [ ]:
# Since we have some metadatas (Title), we need to define a dictionary containing those values
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt'}

In [ ]:
controller.data_store.num_proc=1

In [ ]:
df_result = controller.predict_raw_text(raw_content,topk=3)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
df_result

,Review Text,Title,input_ids,attention_mask,pred_Division Name,pred_prob_Division Name,pred_Department Name,pred_prob_Department Name
0,great shirt . this shirt is so comfortable i l...,great shirt,"[18223, 10147, 764, 428, 10147, 318, 523, 6792...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[General, General Petite, Initmates]","[0.64990836, 0.3206482, 0.029443381]","[Tops, Intimate, Jackets]","[0.99345136, 0.0062913294, 0.00021860567]"
